In [1]:
import numpy as np
import pandas as pd
import pickle
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, make_scorer
from vowpalwabbit.sklearn_vw import VWClassifier, VW
import itertools
from sklearn.ensemble import RandomForestClassifier
import datetime

In [2]:
import xgboost as xgb

In [35]:
from imblearn.under_sampling import     RandomUnderSampler

In [3]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [4]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from scipy.sparse import csr_matrix, hstack

In [18]:
#!gsutil cp -r gs://smartandnimble/identifyme "data"

Copying gs://smartandnimble/identifyme/test_sparse_w5.pkl...
Copying gs://smartandnimble/identifyme/train_part_sparse_w5.pkl...              - [1 files][ 42.7 MiB/ 42.7 MiB]                                                
Copying gs://smartandnimble/identifyme/train_sparse_w5.pkl...                   \\ [1 files][ 89.6 MiB/133.5 MiB]                                                || [2 files][133.5 MiB/133.5 MiB]                                                
==> NOTE: You are downloading one or more large file(s), which would            
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

|-- [2 files][229.5 MiB/314.7 MiB]                                                \\ [3 files][314.7 MiB/314.7 MiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use 

In [5]:
%%time
with open('data/identifyme/train_part_sparse_w5.pkl', 'rb') as f:
    X_train = pickle.load(f)
with open('data/identifyme/valid_sparse_w5.pkl', 'rb') as f:
    X_valid = pickle.load(f)
with open('data/identifyme/train_sparse_w5.pkl', 'rb') as f:
    X = pickle.load(f)
with open('data/identifyme/test_sparse_w5.pkl', 'rb') as f:
    X_test = pickle.load(f)
    
y=pd.read_csv("data/identifyme/y_w5.csv", header=None, squeeze=True)
y_train=pd.read_csv("data/identifyme/y_train_w5.csv", header=None, squeeze=True)
y_valid=pd.read_csv("data/identifyme/y_valid_w5.csv", header=None, squeeze=True)

CPU times: user 2.08 s, sys: 696 ms, total: 2.77 s
Wall time: 4.61 s


In [30]:
train_set = hstack([csr_matrix(np.transpose([y_train])), X_train]).tocsr()

In [54]:
def get_bootstrap_samples(data, n_samples):
    samples = []
    indices = np.random.randint(0, data.shape[0], (n_samples, data.shape[0]))
    print indices
    [samples.append(data[indices[i]]) for i in range(n_samples)]
    return samples

In [55]:
samples = get_bootstrap_samples(train_set, 11)

[[ 72108 110871  63293 ..., 112131  19045  67237]
 [ 72051  64537  79100 ..., 109830 104600  38297]
 [106652 116289  41255 ..., 110036  43274  40767]
 ..., 
 [ 81935  93970 103658 ...,  40742   3649   7070]
 [ 81419  37185  83981 ...,  86996  72211  21418]
 [ 42798 119362  55487 ...,  90735  50941 103300]]


In [58]:
%%time
X_ress = []
y_ress = []
for i in range(len(samples)):
    rus = RandomUnderSampler(random_state=42)
    X_res, y_res = rus.fit_sample(samples[i][:,1:].toarray(), \
                                  np.transpose(samples[i][:,:1].toarray())[0].astype(int).tolist())
    X_res = csr_matrix(X_res)
    X_ress.append(X_res)
    y_ress.append(y_res)

CPU times: user 25min 29s, sys: 42min 26s, total: 1h 7min 55s
Wall time: 1h 7min 55s


In [62]:
%%time
print datetime.datetime.now().strftime('Started on %d, %b %Y - %H:%M')
forests = {}
predictions = {}
for i in range(len(samples)):
    print "Number:", i
    forest = RandomForestClassifier(n_estimators=5000, oob_score=True, n_jobs=4, random_state=7)
    forest.fit(X_ress[i], y_ress[i])
    prediction = forest.predict(X_valid)
    print "Accuracy:", accuracy_score(y_valid, prediction)
    forests[i] = forest
    predictions[i] = [prediction]

print datetime.datetime.now().strftime('Finished on %d, %b %Y - %H:%M')

Started on 12, Jan 2017 - 23:56
Number: 0


JoblibMemoryError: JoblibMemoryError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/usr/local/bin/miniconda/lib/python2.7/runpy.py in _run_module_as_main(mod_name='ipykernel.__main__', alter_argv=1)
    169     pkg_name = mod_name.rpartition('.')[0]
    170     main_globals = sys.modules["__main__"].__dict__
    171     if alter_argv:
    172         sys.argv[0] = fname
    173     return _run_code(code, main_globals, None,
--> 174                      "__main__", fname, loader, pkg_name)
        fname = '/usr/local/bin/miniconda/lib/python2.7/site-packages/ipykernel/__main__.py'
        loader = <pkgutil.ImpLoader instance>
        pkg_name = 'ipykernel'
    175 
    176 def run_module(mod_name, init_globals=None,
    177                run_name=None, alter_sys=False):
    178     """Execute a module's code without importing it

.........................